In [ ]:
# Import required modules
import time
import pandas as pd
from marketwatch import MarketWatch

# Initialize MarketWatch API
username = 'your_username_here'
password = 'your_password_here'
game_name = 'your_game_name_here'

# Initialize MarketWatch
marketwatch = MarketWatch(username, password)

# Function to calculate EMA
def calculate_ema(price_series, periods):
    return price_series.ewm(span=periods, adjust=False).mean()

# Function to calculate RSI
def calculate_rsi(df, periods):
    delta = df['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=periods, min_periods=1).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=periods, min_periods=1).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

# Function to trade based on EMA crossover and RSI
def trade_on_strategies(df, game_name):
    # EMA Crossover
    short_term_ema = df['EMA_20'].iloc[-1]
    long_term_ema = df['EMA_80'].iloc[-1]
    
    # RSI
    rsi = df['RSI_14'].iloc[-1]
    
    if short_term_ema > long_term_ema and rsi < 30:
        print('Conditions met for buying. Attempting to buy stock.')
        marketwatch.buy(game_name, 'AAPL', 100)
    elif short_term_ema < long_term_ema and rsi > 70:
        print('Conditions met for selling. Attempting to sell stock.')
        marketwatch.sell(game_name, 'AAPL', 100)

In [ ]:
# Collect historical data by calling get_price() at intervals
price_data = []
for _ in range(160):
    price = marketwatch.get_price('AAPL')
    price_data.append(price)
    if len(price_data) > 150:
        price_data.pop(0)
    
    df = pd.DataFrame(price_data, columns=['Close'])
    
    for period in [20, 80]:
        ema_column_name = f'EMA_{period}'
        df[ema_column_name] = calculate_ema(df['Close'], period)
    
    df['RSI_14'] = calculate_rsi(df, 14)
    
    if len(df) >= 80:
        trade_on_strategies(df, game_name)
    
    time.sleep(60)